In [1]:
from CC import CC
from DnsVhost import DnsVhost
from MariaDB import MariaDB
from WP import WP
from Partition import Partition

## 객체 ###
cc =  CC("172.16.16.122", "root", "asd123!@")
cc.connect()
dv = DnsVhost() 
mariadb = MariaDB()
wp = WP()
part = Partition()

## 전역변수 ##
ZONE_FILE = ""
IPADDR = ""


###################################################
#              HTTP,DNS -DnsVhost.py              #
###################################################
# 나중에 수정할 필요 있음. DB에 맞춰..
#---------------------HTTP-----------------------#
## 1. http 설치, vhost 수정, 재시작 , DNS설치 및 설정까지 : 밑의 mod_http_R 함수 사용
def install_and_setting_http() :
    cc.run(dv.install_http())
    mod_http_and_dns()

## 2. vhost 수정, 재시작
def mod_http_and_dns():
    cc.run(dv.set_vhostconf())
    set_httpd_and_dns()
    cc.run(dv.restart_http())

## 3. vhost 수정, 만약 zone파일이 존재하면 vhost의 내용을 반영하고 3차도메인을 수정합니다.
def set_httpd_and_dns() :
    vhost_conf = dv.getter_VHOST_CONF()
    print(f"현재 당신의 vhost.conf 파일은 {vhost_conf}입니다.")
    while(True) :
        cc.run(dv.show_vhostconf())
        num = int(input("(1) vhost 추가하기, (0) vhost 파일 수정 종료"))
        if(num == 0) :
            print("vhost 파일 수정을 종료합니다")
            break;
        cc.run(dv.mod_vhostconf())
    cc.run(dv.restart_http())

    print("httpd와 관련한 vhost내용은 성공적으로 반영되었습니다. 이에따른 DNS 내용도 수정하겠습니다")

    ## DNS 내용들임
    install_and_setting_dns()
    #cc.dv.set_dns_Third_domain()

        
#---------------------DNS-----------------------#
## 1. DNS 설치 및 ZONE 파일 수정, vhost에 따른 3차도메인 자동 업데이트
def install_and_setting_dns():
    cc.run(dv.install_named())
    cc.run(dv.set_named_conf())
    cc.run(dv.set_named_rfc())
    cc.run(dv.mk_zone_file())
    
    cc.run(dv.set_zone_file())
    cc.run(dv.set_dns_Third_domain())
    cc.run(dv.start_dns())

#---------------------실행부-----------------------#
#install_and_setting_http()



###################################################
#            MariaDB,PHP -MariaDB.py              #
###################################################
## 1. mariaDB와 php 둘 다 설치함
def install_mariaDB_and_php():
    cc.run(mariadb.install_packages())
    cc.run(mariadb.install_apache_and_php())
    cc.run(mariadb.install_mariaDB())
    cc.run(mariadb.init_mariaDB())
    cc.run(mariadb.install_php_my_admin())
    cc.run(mariadb.set_phpMyadmin_conf())
#---------------------실행부-----------------------#
#install_mariaDB_and_php()


###################################################
#            Partition -Partition.py              #
###################################################
def mk_partition():
    cc.run("lsblk")
    cc.run(part.make_partition())
    cc.run(part.format_partition())
    cc.run(part.mount())
    cc.run(part.permanent_mount_config_remote())


#---------------------실행부-----------------------#
#mk_partition()



###################################################
#                 WP,PHPMYSQL -WP.py              #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_wp():
    wp.install_prev_packages()
    wp.install_and_move_dir()
    wp.set_wp_config_p()

실행
DnsVhost init start
MariaDB init start
WP init 시작
NAME        MAJ:MIN RM  SIZE RO TYPE MOUNTPOINTS
sda           8:0    0   10M  0 disk 
└─sda1        8:1    0    9M  0 part 
sdb           8:16   0   10M  0 disk 
sdc           8:32   0   10M  0 disk 
sdd           8:48   0   10M  0 disk 
sr0          11:0    1  1.8G  0 rom  
nvme0n1     259:0    0   20G  0 disk 
├─nvme0n1p1 259:1    0    1G  0 part /boot
└─nvme0n1p2 259:2    0   19G  0 part 
  ├─rl-root 253:0    0   17G  0 lvm  /
  └─rl-swap 253:1    0    2G  0 lvm  [SWAP]



무엇을 파티션할건지 적으세요 (ex. /dev/sda):  /dev/sdb


spawn fdisk /dev/sdb

Welcome to fdisk (util-linux 2.37.4).
Changes will remain in memory only, until you decide to write them.
Be careful before using the write command.

Device does not contain a recognized partition table.
Created a new DOS disklabel with disk identifier 0x7e9b44c1.

Command (m for help): n
Partition type
   p   primary (0 primary, 0 extended, 4 free)
   e   extended (container for logical partitions)
Select (default p): 

Using default response p.
Partition number (1-4, default 1): 
First sector (2048-20479, default 2048): 
Last sector, +/-sectors or +/-size{K,M,G,T,P} (2048-20479, default 20479): 

Created a new partition 1 of type 'Linux' and of size 9 MiB.

Command (m for help): w
The partition table has been altered.
Calling ioctl() to re-read partition table.
Syncing disks.

NAME        MAJ:MIN RM  SIZE RO TYPE MOUNTPOINTS
sda           8:0    0   10M  0 disk 
└─sda1        8:1    0    9M  0 part 
sdb           8:16   0   10M  0 disk 
└─sdb1        8:17   0   

아까 생성한 파티션을 입력하세요(ex. /dev/sda1):  /dev/sdb1


mke2fs 1.46.5 (30-Dec-2021)
Creating filesystem with 9216 1k blocks and 2304 inodes
Filesystem UUID: ece75fca-f142-43cc-9704-32586260b474
Superblock backups stored on blocks: 
	8193

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (1024 blocks): done
Writing superblocks and filesystem accounting information: done

NAME FSTYPE FSVER LABEL UUID                                   FSAVAIL FSUSE% MOUNTPOINTS
sda                                                                           
└─sda1
     ext4   1.0         c7561b86-6218-4559-9b53-d43926502d37                  
sdb                                                                           
└─sdb1
     ext4   1.0         ece75fca-f142-43cc-9704-32586260b474                  
sdc                                                                           
sdd                                                                           
sr0  iso966 Jolie Rocky-

마운트 할 경로를 입력하세요 (ex. /mnt/test):  /mnt/sdb1


lost+found

fstab에 새 항목 추가함: /dev/sdb1 /mnt/sdb1 ext4    defaults,usrquota        0 0


